In [1]:
import torch
import torch.nn as nn
import core
from core.rnn import RnnEncoder

from torchvision import datasets, transforms
from torch import nn
from torch.nn import functional as F
from torch.distributions import Categorical
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import random
import numpy as np

import csv
import re
import os
import unicodedata
import codecs
import itertools
from xml.dom import minidom
import xml.etree.ElementTree as ET


from pylab import rcParams
rcParams['figure.figsize'] = 5, 10

# For convenince and reproducibility, we set some EGG-level command line arguments here
opts = core.init(params=['--random_seed=7', # will initialize numpy, torch, and python RNGs
                         '--lr=1e-3',   # sets the learning rate for the selected optimizer 
                         '--batch_size=32',
                         '--optimizer=adam'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(0)
np.random.seed(0)

In [2]:
class Vocabulary:
    def __init__(self):
        self.attribute2index = {}
        self.attribute2count = {}
        self.index2attribute = {}
        self.num_attributes = 0
        self.concept_list = []
        self.concept2vector = {}
        self.num_concepts = 0
        
    def schredderConcept(self, animal):
        i = 0
        attributes= []
        for node in animal.childNodes:
            i+=1
            if i%2 == 0:
                for attribute in node.childNodes[0].data.split('\n'):
                    if attribute != '':
                        attributes.append(attribute.strip())
        return attributes
        
    def addConcept(self, animal):
        self.num_concepts += 1
        self.concept_list.append(animal)
        attributes = self.schredderConcept(animal)
        for attribute in attributes:
            self.addAttr(attribute)                        
                        
            
    def addAttr(self,attribute):
        if attribute not in self.attribute2index:
            self.attribute2index[attribute] = self.num_attributes
            self.attribute2count[attribute] = 1
            self.index2attribute[self.num_attributes] = attribute
            self.num_attributes += 1
        else:
            self.attribute2count[attribute] += 1
            
    def parseConcept(self, animal):
        if animal.attributes['name'].value not in self.concept2vector:
            attributes = self.schredderConcept(animal)
            vector = torch.zeros(self.num_attributes)
            for attribute in attributes:
                vector[self.attribute2index[attribute]]=1
            self.concept2vector[animal.attributes['name'].value] = vector
        else:
            vector = self.concept2vector[animal.attributes['name'].value]
        return vector

In [3]:
class ConceptDataSet(Dataset):
    def __init__(self, voc, num_options):
        super(ConceptDataSet, self).__init__()
        self.voc = voc
        self.num_options = num_options
        
    def __len__(self):
        return self.voc.num_concepts
    
    def __getitem__(self, idx):
        concept = self.voc.concept_list[idx]
        sender_input = self.voc.parseConcept(concept)
        
        commute = torch.randint(0,self.num_options,[1])
        
        receiver_input = []
        for option in range(self.num_options):
            if option==commute:
                receiver_input.append(sender_input)
            else:
                x = list(range(1,idx)) + list(range(idx+1, self.voc.num_concepts))
                r = random.choice(x)
                c = self.voc.concept_list[r]
                receiver_input.append(self.voc.parseConcept(c))      
        sample = [sender_input, commute, receiver_input]
        return sample

In [4]:
concept_list = Vocabulary()

def addFile(name):
    file_name = name + "_structured_final.us.xml"
    file = minidom.parse(os.path.join(os.path.join("visa_dataset", "US"), file_name))
    concepts = file.getElementsByTagName('concept')

    for concept in concepts:
        concept_list.addConcept(concept)

addFile("ANIMALS")
addFile("APPLIANCES")
addFile("ARTIFACTS")
addFile("CLOTHING")
addFile("CONTAINER")
addFile("DEVICE")
addFile("FOOD")
addFile("HOME")
addFile("INSTRUMENTS")
addFile("MATERIAL")
addFile("PLANTS")
addFile("STRUCTURES")
addFile("TOOLS")
addFile("TOYS")
addFile("VEHICLES")
addFile("WEAPONS")

print(concept_list.num_concepts)
print(concept_list.num_attributes)

509
597


In [5]:
vocab_size = 100
max_len = 5
num_options = 5
batch_size = 32

In [6]:
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

train_loader = torch.utils.data.DataLoader(
        ConceptDataSet(concept_list, num_options),
           batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
        ConceptDataSet(concept_list, num_options),
           batch_size=batch_size, shuffle=False, **kwargs)

In [7]:
class PrePro(nn.Module):
    def __init__(self, hidden_size, voc):
        super(PrePro, self).__init__()
        self.pre_linguistic = nn.Linear(voc.num_attributes, hidden_size)
        self.act = nn.Sigmoid()
        
    def forward(self, input_concept):
        raw = self.pre_linguistic(input_concept)
        output = self.act(raw)
        return output
    
    
class Receiver(nn.Module):
    def __init__(self, hidden_size, num_options, voc):
        super(Receiver, self).__init__()
        self.encoder = PrePro(hidden_size,voc)
        self.num_options = num_options
        
    def forward(self, channel_input, receiver_input=None):
        c_list =  []
        for i in range(self.num_options):
            c_list.append(self.encoder(receiver_input[i]))
        concepts = torch.stack(c_list)  
        concepts= concepts.transpose(0,1)

        channel_input = channel_input[:,:,None]

        dotproduct = torch.bmm(concepts, channel_input)
        dotproduct = dotproduct.squeeze(dim=-1)

        pre_logits = F.log_softmax(dotproduct, dim=1)

        distr = Categorical(logits=pre_logits)

        if self.training:
            sample = distr.sample()
        else:
            sample = pre_logits.argmax(dim=1)
        log_prob = distr.log_prob(sample)

        entropy = distr.entropy()

        return sample, log_prob, entropy
    
class EasyGuesser(nn.Module):
    def __init__(self, hidden_size):
        super(EasyGuesser, self).__init__()
        self.empathy = nn.Linear(hidden_size, hidden_size)

    def forward(self, hidden):
        encoded = self.empathy(hidden)
        return torch.sigmoid(encoded)

In [8]:
#loss um prediction mit Realität zu vergleichen
def loss(_sender_input, _message, _receiver_input, receiver_output, labels):
    acc = (labels.t() == receiver_output).float() - (labels.t() != receiver_output).float()
    return -acc, {'acc': (acc.mean().item()+1)/2}

#loss um prediction mit Realität zu vergleichen
def auxiliary_loss(receiver_thoughts, _message, _receiver_input, guesser_output, _labels, weight=0.2):
    loss = F.l1_loss(guesser_output, receiver_thoughts, reduction='none').mean(dim=1)
    loss *= weight
    return loss, {}

In [9]:
# we start by defining the RNN parameters
hidden_size = 50
emb_size = 50

In [10]:
sender = PrePro(hidden_size, concept_list)
receiver = Receiver(hidden_size, num_options, concept_list)

sender_rnn = core.RnnSenderReinforce(sender, vocab_size, emb_size, hidden_size,
                                   cell="gru", max_len= max_len)
receiver_rnn = core.RnnReceiverReinforce(receiver, vocab_size, emb_size,
                    hidden_size, cell="gru")

auxiliary_rnn = EasyGuesser(hidden_size)

game_rnn = core.SenderReceiverRnnReinforceAuxiliaryTask(sender_rnn, receiver_rnn, auxiliary_rnn, loss, auxiliary_loss, 
                                           sender_entropy_coeff=0.015, 
                                           receiver_entropy_coeff=0.0)

In [11]:
optimizer = torch.optim.Adam([
        {'params': game_rnn.sender.parameters(), 'lr': 1e-3},
        {'params': game_rnn.receiver.parameters(), 'lr': 1e-2},
        {'params': game_rnn.guesser.parameters(), 'lr': 1e-2}
    ])

In [12]:
trainer = core.Trainer(game=game_rnn, optimizer=optimizer, train_data=train_loader,
                           validation_data=test_loader)
common_opts = core.util.get_opts()

torch.manual_seed(0)
np.random.seed(0)

trainer.train(2000)

test: epoch 1, loss 1.1651017665863037, aux_loss 0.05392967909574509,  {'acc': 0.20332704670727253, 'loss': 1.1651020050048828, 'sender_entropy': 3.6687068790197372, 'receiver_entropy': 0.86431834846735, 'original_loss': 0.5933459065854549, 'mean_length': 5.0}
test: epoch 2, loss 0.37502020597457886, aux_loss 0.047763798385858536,  {'acc': 0.1953125, 'loss': 0.3750203214585781, 'sender_entropy': 3.6698273569345474, 'receiver_entropy': 1.2244441509246826, 'original_loss': 0.609375, 'mean_length': 5.0}
test: epoch 3, loss 1.3145731687545776, aux_loss 0.06250715255737305,  {'acc': 0.2248787716962397, 'loss': 1.3145731762051582, 'sender_entropy': 3.6681203097105026, 'receiver_entropy': 1.239837385714054, 'original_loss': 0.5502424566075206, 'mean_length': 5.0}
test: epoch 4, loss 0.23252534866333008, aux_loss 0.05807145684957504,  {'acc': 0.19228178879711777, 'loss': 0.23252521455287933, 'sender_entropy': 3.667283311486244, 'receiver_entropy': 1.1157708875834942, 'original_loss': 0.6154364

test: epoch 33, loss 0.856806218624115, aux_loss 0.045380737632513046,  {'acc': 0.2113415952771902, 'loss': 0.8568060211837292, 'sender_entropy': 3.58693490922451, 'receiver_entropy': 0.8593230620026588, 'original_loss': 0.5773168094456196, 'mean_length': 5.0}
test: epoch 34, loss 1.3079750537872314, aux_loss 0.06430798768997192,  {'acc': 0.22703394386917353, 'loss': 1.3079751059412956, 'sender_entropy': 3.576221287250519, 'receiver_entropy': 0.43006752617657185, 'original_loss': 0.545932112261653, 'mean_length': 5.0}
test: epoch 35, loss 0.12234394997358322, aux_loss 0.0456840880215168,  {'acc': 0.18440194055438042, 'loss': 0.12234386429190636, 'sender_entropy': 3.5595239996910095, 'receiver_entropy': 1.1056633219122887, 'original_loss': 0.6311961188912392, 'mean_length': 5.0}
test: epoch 36, loss 0.948374330997467, aux_loss 0.044680532068014145,  {'acc': 0.21646012924611568, 'loss': 0.9483743831515312, 'sender_entropy': 3.5470525920391083, 'receiver_entropy': 0.9163988083600998, 'ori

test: epoch 64, loss 0.15066003799438477, aux_loss 0.06273361295461655,  {'acc': 0.2068965518847108, 'loss': 0.15066002681851387, 'sender_entropy': 3.565052881836891, 'receiver_entropy': 0.33704717457294464, 'original_loss': 0.5862068962305784, 'mean_length': 2.865369066596031}
test: epoch 65, loss 0.4385448098182678, aux_loss 0.05970421060919762,  {'acc': 0.2285829745233059, 'loss': 0.4385448098182678, 'sender_entropy': 3.578834667801857, 'receiver_entropy': 0.3719022860750556, 'original_loss': 0.5428340509533882, 'mean_length': 2.4181707948446274}
test: epoch 66, loss 0.14685142040252686, aux_loss 0.06035829707980156,  {'acc': 0.2051454745233059, 'loss': 0.14685144275426865, 'sender_entropy': 3.5855948477983475, 'receiver_entropy': 0.37543814070522785, 'original_loss': 0.5897090509533882, 'mean_length': 2.4375}
test: epoch 67, loss 0.1930798888206482, aux_loss 0.06082405522465706,  {'acc': 0.2166621768847108, 'loss': 0.1930798925459385, 'sender_entropy': 3.5824791938066483, 'receiver

test: epoch 95, loss 0.6612029671669006, aux_loss 0.024226155132055283,  {'acc': 0.2386853452771902, 'loss': 0.6612030267715454, 'sender_entropy': 3.5072453767061234, 'receiver_entropy': 0.632164953276515, 'original_loss': 0.5226293094456196, 'mean_length': 1.048828125}
test: epoch 96, loss 0.5604778528213501, aux_loss 0.02359182946383953,  {'acc': 0.22285829670727253, 'loss': 0.560477901250124, 'sender_entropy': 3.4998688399791718, 'receiver_entropy': 0.6285402178764343, 'original_loss': 0.5542834065854549, 'mean_length': 1.05859375}
test: epoch 97, loss 0.5931082963943481, aux_loss 0.02270130254328251,  {'acc': 0.22090517170727253, 'loss': 0.593108382076025, 'sender_entropy': 3.4896414279937744, 'receiver_entropy': 0.6024970039725304, 'original_loss': 0.5581896565854549, 'mean_length': 1.060546875}
test: epoch 98, loss 0.7054921984672546, aux_loss 0.024197842925786972,  {'acc': 0.23828125, 'loss': 0.7054921146482229, 'sender_entropy': 3.483418107032776, 'receiver_entropy': 0.63431673

test: epoch 125, loss 1.6993836164474487, aux_loss 0.04219283536076546,  {'acc': 0.3707570042461157, 'loss': 1.6993837244808674, 'sender_entropy': 2.6596556045114994, 'receiver_entropy': 0.4201762229204178, 'original_loss': 0.25848599150776863, 'mean_length': 2.5847925692796707}
test: epoch 126, loss 1.5656414031982422, aux_loss 0.044094450771808624,  {'acc': 0.3922413792461157, 'loss': 1.5656412607058883, 'sender_entropy': 2.731768898665905, 'receiver_entropy': 0.41150783747434616, 'original_loss': 0.21551724150776863, 'mean_length': 3.0603448301553726}
test: epoch 127, loss 1.3452726602554321, aux_loss 0.04299468547105789,  {'acc': 0.4011988155543804, 'loss': 1.3452726732939482, 'sender_entropy': 2.680097311735153, 'receiver_entropy': 0.4056160170584917, 'original_loss': 0.19760236889123917, 'mean_length': 3.1490436494350433}
test: epoch 128, loss 1.531006097793579, aux_loss 0.044644344598054886,  {'acc': 0.3748653018847108, 'loss': 1.531006084755063, 'sender_entropy': 2.642756715416

...

test: epoch 1872, loss -0.7132911086082458, aux_loss 0.05238088220357895,  {'acc': 0.865234375, 'loss': -0.713291060179472, 'sender_entropy': 0.06306210224283859, 'receiver_entropy': 0.19755715876817703, 'original_loss': -0.73046875, 'mean_length': 3.8275188505649567}
test: epoch 1873, loss -0.7416020035743713, aux_loss 0.052549850195646286,  {'acc': 0.8845635782927275, 'loss': -0.741601999849081, 'sender_entropy': 0.059555742132943124, 'receiver_entropy': 0.1708783581852913, 'original_loss': -0.7691271565854549, 'mean_length': 3.8275188505649567}
test: epoch 1874, loss -0.6700286865234375, aux_loss 0.05267013609409332,  {'acc': 0.8259698282927275, 'loss': -0.6700286958366632, 'sender_entropy': 0.057158310926752165, 'receiver_entropy': 0.20006950735114515, 'original_loss': -0.6519396565854549, 'mean_length': 3.8413927853107452}
test: epoch 1875, loss -0.6893307566642761, aux_loss 0.053247302770614624,  {'acc': 0.8433459047228098, 'loss': -0.6893307846039534, 'sender_entropy': 0.0555188

test: epoch 1902, loss -0.696138858795166, aux_loss 0.05177690088748932,  {'acc': 0.8626751080155373, 'loss': -0.6961388820782304, 'sender_entropy': 0.06606406171340495, 'receiver_entropy': 0.2012380100786686, 'original_loss': -0.7253502160310745, 'mean_length': 3.8097386956214905}
test: epoch 1903, loss -0.7183817028999329, aux_loss 0.05169326066970825,  {'acc': 0.86328125, 'loss': -0.7183816879987717, 'sender_entropy': 0.06475467933341861, 'receiver_entropy': 0.2060783337801695, 'original_loss': -0.7265625, 'mean_length': 3.8138469755649567}
test: epoch 1904, loss -0.7109771370887756, aux_loss 0.051987722516059875,  {'acc': 0.8492052797228098, 'loss': -0.710977103561163, 'sender_entropy': 0.06276153749786317, 'receiver_entropy': 0.2295312760397792, 'original_loss': -0.6984105594456196, 'mean_length': 3.8294719755649567}
test: epoch 1905, loss -0.6999289393424988, aux_loss 0.051836591213941574,  {'acc': 0.8488011844456196, 'loss': -0.6999290008097887, 'sender_entropy': 0.0626809802488

test: epoch 1932, loss -0.6985087990760803, aux_loss 0.05147743970155716,  {'acc': 0.8366783410310745, 'loss': -0.6985087636858225, 'sender_entropy': 0.056809182453434914, 'receiver_entropy': 0.2059809761121869, 'original_loss': -0.673356682062149, 'mean_length': 3.8357354402542114}
test: epoch 1933, loss -0.6778395175933838, aux_loss 0.052872687578201294,  {'acc': 0.8331761844456196, 'loss': -0.6778395362198353, 'sender_entropy': 0.06337755179265514, 'receiver_entropy': 0.23521536029875278, 'original_loss': -0.6663523688912392, 'mean_length': 3.8415948152542114}
test: epoch 1934, loss -0.6915280818939209, aux_loss 0.05267846956849098,  {'acc': 0.8429418094456196, 'loss': -0.69152807071805, 'sender_entropy': 0.05794364836765453, 'receiver_entropy': 0.2198597709648311, 'original_loss': -0.6858836188912392, 'mean_length': 3.8513604402542114}
test: epoch 1935, loss -0.7160336375236511, aux_loss 0.05233476310968399,  {'acc': 0.8390355594456196, 'loss': -0.7160336505621672, 'sender_entropy'

test: epoch 1962, loss -0.6745588779449463, aux_loss 0.05281340330839157,  {'acc': 0.8181573282927275, 'loss': -0.67455886118114, 'sender_entropy': 0.06778375641442835, 'receiver_entropy': 0.25183939654380083, 'original_loss': -0.6363146565854549, 'mean_length': 3.8767510652542114}
test: epoch 1963, loss -0.715871274471283, aux_loss 0.052598774433135986,  {'acc': 0.8585668094456196, 'loss': -0.7158713452517986, 'sender_entropy': 0.0670212694676593, 'receiver_entropy': 0.20297984359785914, 'original_loss': -0.7171336188912392, 'mean_length': 3.8828125}
test: epoch 1964, loss -0.747320830821991, aux_loss 0.05248361453413963,  {'acc': 0.8611260782927275, 'loss': -0.7473209016025066, 'sender_entropy': 0.07134171086363494, 'receiver_entropy': 0.20172940334305167, 'original_loss': -0.7222521565854549, 'mean_length': 3.884765625}
test: epoch 1965, loss -0.7347221374511719, aux_loss 0.05250238999724388,  {'acc': 0.861328125, 'loss': -0.7347220927476883, 'sender_entropy': 0.06963377119973302, '

test: epoch 1992, loss -0.7193147540092468, aux_loss 0.05180723965167999,  {'acc': 0.8626751080155373, 'loss': -0.7193147540092468, 'sender_entropy': 0.08037783985491842, 'receiver_entropy': 0.22149574011564255, 'original_loss': -0.7253502160310745, 'mean_length': 3.884765625}
test: epoch 1993, loss -0.7103552222251892, aux_loss 0.05205439776182175,  {'acc': 0.8507543094456196, 'loss': -0.710355207324028, 'sender_entropy': 0.07899468927644193, 'receiver_entropy': 0.215596963185817, 'original_loss': -0.7015086188912392, 'mean_length': 3.884765625}
test: epoch 1994, loss -0.7203655242919922, aux_loss 0.05186022073030472,  {'acc': 0.8689385782927275, 'loss': -0.7203655671328306, 'sender_entropy': 0.07906363729853183, 'receiver_entropy': 0.20355834835208952, 'original_loss': -0.7378771565854549, 'mean_length': 3.884765625}
test: epoch 1995, loss -0.7145376205444336, aux_loss 0.05164285749197006,  {'acc': 0.8630792032927275, 'loss': -0.7145376801490784, 'sender_entropy': 0.07728594657965004